In [9]:
# import libraries (you may add additional imports but you may not have to)

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt


In [21]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'


--2024-06-11 12:05:54--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  87.3MB/s    in 0.3s    

2024-06-11 12:05:54 (87.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [36]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [39]:
# Cell 4
# Remove users with less than 200 ratings and books with less than 100 ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]


In [49]:

# Merge books and ratings
df = pd.merge(df_books, df_ratings, on='isbn')

# Pivot table to create user-item matrix
book_ratings = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert the DataFrame to a CSR matrix
book_ratings_matrix = csr_matrix(book_ratings.values)

# Fit Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_ratings_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [52]:
def get_recommends(book = ""):
    distances, indices = model_knn.kneighbors(book_ratings.loc[book].values.reshape(1, -1), n_neighbors=6)
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([book_ratings.index[indices.flatten()[i]], distances.flatten()[i]])
    return [book, recommended_books]

In [60]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

# Cell 7 (Testing function)
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
    print(recommends[0] + ",")
    print("[")
    for book, distance in recommends[1]:
        print(f"  ['{book}', {distance}],")
    print("]")
    if recommends[0] != "The Queen of the Damned (Vampire Chronicles (Paperback))":
        test_pass = False
    recommended_books = ["Catch 22", 'The Witching Hour (Lives of the Mayfair Witches)', 'Interview with the Vampire', 'The Tale of the Body Thief (Vampire Chronicles (Paperback))', 'The Vampire Lestat (Vampire Chronicles, Book II)']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2):
        if recommends[1][i][0] != recommended_books[i] or abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = True
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['Interview with the Vampire', 0.73450685], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Catch 22', 0.7939835]]]
The Queen of the Damned (Vampire Chronicles (Paperback)),
[
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178411602973938],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
  ['Interview with the Vampire', 0.7345068454742432],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
  ['Catch 22', 0.793983519077301],
]
You passed the challenge! 🎉🎉🎉🎉🎉
